In [97]:
import pandas as pd
import numpy as np
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline

# Reading file

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df = df[df.Stock == 'A']

In [4]:
df.describe()

,Open,High,Low,Close,Volume,Adj Close,Average,Return,Max,Min,Day,y
count,1259.000000,1259.000000,1259.000000,1259.000000,1.259000e+03,1259.000000,1259.000000,1258.000000,1259.0,1259.0,1259.000000,1258.000000
mean,0.622552,0.626594,0.624178,0.624213,4.357144e+06,20.688205,0.626183,0.000429,1.0,5.0,0.336643,2.018283
std,0.220899,0.223228,0.221613,0.221125,2.021792e+06,4.335443,0.223169,0.022797,0.0,0.0,0.233183,0.962536
min,0.000000,0.000000,0.000000,0.000000,9.693000e+05,8.433733,0.000000,-0.108450,1.0,5.0,0.000000,0.000000
25%,0.540210,0.543814,0.537979,0.538331,3.036550e+06,19.033525,0.542720,-0.009621,1.0,5.0,0.166667,2.000000
50%,0.676960,0.682818,0.678196,0.679444,3.967500e+06,21.728388,0.681292,0.001001,1.0,5.0,0.333333,2.000000
75%,0.781812,0.790378,0.782299,0.782734,5.117300e+06,23.824879,0.786014,0.011127,1.0,5.0,0.500000,3.000000
max,1.000000,1.000000,1.000000,1.000000,2.068870e+07,28.419782,1.000000,0.138790,1.0,5.0,0.666667,4.000000


In [5]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Stock,Average,Return,Max,Min,Day,y
0,2006-01-03,0.710553,0.707904,0.705324,0.714383,5307000,21.399132,A,0.711972,NaN,1,5,0.166667,NaN
1,2006-01-04,0.715643,0.716495,0.723974,0.717436,4195800,21.456621,A,0.720884,0.002687,1,5,0.333333,2.0
2,2006-01-05,0.712250,0.738488,0.726687,0.747286,4835400,22.018748,A,0.733739,0.026198,1,5,0.500000,3.0
3,2006-01-06,0.747879,0.749485,0.748389,0.753392,6146300,22.133729,A,0.752507,0.005222,1,5,0.666667,2.0
4,2006-01-09,0.752969,0.749828,0.759580,0.750000,4082800,22.069850,A,0.755849,-0.002886,1,5,0.000000,2.0


# X & Y

In [128]:
k=1 #Max lags

X = pd.DataFrame()
X.loc[:,'Day'] = df.loc[:,'Day'].iloc[0:-k].reset_index(drop=True)
y = df['y'].iloc[k:].reset_index(drop=True)
y2 = df['Return'].iloc[k:].reset_index(drop=True)

for i in range(k):
        X.loc[:,('Lag %i' % (i+1))] = df.loc[:,'Average'].iloc[k-1-i:-1-i].reset_index(drop=True)

# X with Open, High, Low and Close

In [7]:
X = pd.DataFrame()
X = df.loc[:,['Open','High','Low','Close']].iloc[0:-k].reset_index(drop=True)

# X with 0 and 1 if increasing or not

In [134]:
X= pd.DataFrame(np.random.binomial(1,0.5,size=(y.shape[0],4)))

# Spliting

In [135]:
split_var = int(X.shape[0]/4*3)
X_train, X_test, y_train, y_test = [X.iloc[:split_var,:],X.iloc[split_var:,:],y.iloc[:split_var],y.iloc[split_var:]]

In [136]:
X_train.head()

,0,1,2,3
0,0,0,0,1
1,1,1,1,1
2,0,0,0,1
3,1,0,1,0
4,0,1,1,1


In [137]:
y_train.head()

0    2.0
1    3.0
2    2.0
3    2.0
4    3.0
Name: y, dtype: float64

# Table

lags | C |n_iter| batch_size | hidden_units | learning_rate | Scr_training | Scr_test
--- | --- | ---
1|1|10|90|100 | .1 |0.479321314952|0.51746031746
1|1|10|90|100| .01|0.479321314952|0.51746031746
2|1|10|90|100| .01|0.478768577495|0.51746031746
3|1|10|90|100|.01|0.478768577495|0.519108280255
4|1|10|90|100|.01|0.47821466525|0.519108280255
5|1|10|90|100|.01|0.477659574468|0.519108280255
5|1|10|90|100|.001|0.477659574468|0.519108280255
5|100|10|90|100|.001|0.479321314952|0.51746031746
5|1000|10|90|100|.001|0.479321314952|0.51746031746

# Running RBM-logistic

In [138]:
# for rbm
learning_rate = 0.001
hidden_units = 100
batch_size = 90
n_iter = 10

# for logistic
C = 100

In [139]:
logistic = LogisticRegression(C=C)
mlp = MLPClassifier()
rbm0 = BernoulliRBM(random_state=0, verbose=False)
rbm1 = BernoulliRBM(random_state=0, verbose=False)

classifier0 = Pipeline([('rbm', rbm0),('mlp',mlp)])
classifier1 = Pipeline([('rbm', rbm1),('logistic',logistic)])

rbm0.learning_rate = learning_rate
rbm0.n_components=hidden_units
rbm0.batch_size = batch_size
rbm0.n_iter = n_iter
rbm1.learning_rate = learning_rate
rbm1.n_components=hidden_units
rbm1.batch_size = batch_size
rbm1.n_iter = n_iter

mlp.hidden_layer_sizes = (100,)

classifier0.fit(X=X_train, y=y_train);
classifier1.fit(X=X_train, y=y_train);

print("Score for RBM-MLP")
print("score on training: ",classifier0.score(X_train,y_train))
print("score on test: ",classifier0.score(X_test,y_test))
print("\nScore for RBM-logistic")
print("score on training: ",classifier1.score(X_train,y_train))
print("score on test: ",classifier1.score(X_test,y_test))
print("\nDifferences in the score between both models")
print("For training: ", classifier0.score(X_train,y_train)-classifier1.score(X_train,y_train))
print("For Testing: ", classifier0.score(X_test,y_test)-classifier1.score(X_test,y_test))

Score for RBM-MLP
score on training:  0.479321314952
score on test:  0.51746031746

Score for RBM-logistic
score on training:  0.479321314952
score on test:  0.51746031746

Differences in the score between both models
For training:  0.0
For Testing:  0.0


In [117]:
l = LogisticRegression(solver='newton-cg', multi_class='multinomial')
l.fit(X_train,y_train);
print("score on training: ",l.score(X_train,y_train))
print("score on test: ",l.score(X_test,y_test))

score on training:  0.478260869565
score on test:  0.51746031746
